<a href="https://colab.research.google.com/github/Umerfarooq122/Text-Summarization-by-Fien-tuning-a-pre-trained-transformer-PEGASUS-from-hugging-face-library/blob/main/Copy_of_Text_Summarization_using_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Text Summarization Using Pre-training with Extracted Gap-sentences for Abstractive Summarization (PEGASUS) from Huggingface**

#### **Environment Setup:**

In [1]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_metric py7zr -q

#!pip install numpy==1.24.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8

In [2]:
!pip install datasets --upgrade
!pip install evaluate --upgrade
!pip install rouge_score --upgrade
!pip install transformers --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=d174b5bc45d070e6386330c61741bf2f1e22cc18958dc95e400160edbdf2f047
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 114.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.

In [3]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
from evaluate import load
import torch
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

#### **Loading PEGASUS:**

In [5]:
model_large = "google/pegasus-large"
tokenizer_large = AutoTokenizer.from_pretrained(model_large)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [6]:
model_pega = AutoModelForSeq2SeqLM.from_pretrained(model_large).to(device)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

#### **Loading PubMED Dataset:**

In [7]:
dataset_science = load_dataset("ccdv/pubmed-summarization")
dataset_science

README.md:   0%|          | 0.00/3.80k [00:00<?, ?B/s]

train-00000-of-00005.parquet:   0%|          | 0.00/210M [00:00<?, ?B/s]

train-00001-of-00005.parquet:   0%|          | 0.00/208M [00:00<?, ?B/s]

train-00002-of-00005.parquet:   0%|          | 0.00/207M [00:00<?, ?B/s]

train-00003-of-00005.parquet:   0%|          | 0.00/211M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/58.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/119924 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6633 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6658 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 119924
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6633
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6658
    })
})

#### **Splitting the Dataset:**

In [8]:
split_lengths = [len(dataset_science[split]) for split in dataset_science]
split_lengths

[119924, 6633, 6658]

In [9]:
print(f"Features: {dataset_science['train'].column_names}")

Features: ['article', 'abstract']


In [ ]:
print("\nArticle:")
print(dataset_science["train"][1]['article'])
print("\nAbstract:")
print(dataset_science["train"][1]["abstract"])


#### **Test Running Raw Model:**

In [23]:
dialogue = dataset_science["train"][1]["article"]
summary = dataset_science["train"][1]["abstract"]
dialogue

'it occurs in more than 50% of patients and may reach 90% in certain types of cancers , especially in patients undergoing chemotherapy and/or radiation therapy.1 anemia is defined as an inadequate circulating level of hemoglobin ( hb ) ( hb < 12 g / dl ) and may arise as a result of the underlying disease , bleeding , poor nutrition , chemotherapy , or radiation therapy . \n preliminary studies suggest that survival and loco - regional control after radiation therapy , especially in head and neck cancers , may be compromised by anemia.24 anemia often worsens symptoms such as fatigue , weakness , and dyspnea , and thus may have a negative effect on quality of life ( qol ) and performance status in patients with cancer . \n thus , to improve physical functioning , qol , and prognosis in patients with cancer , it would be reasonable to take a proactive approach in identifying populations who need treatment for cancer - associated anemia ( caa ) and provide timely management . \n blood tra

In [35]:
'''
pipe = pipeline("summarization", model=model_pega, tokenizer=tokenizer_large, device='cpu')
'''


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [33]:
import os
# Set the environment variable to enable more verbose error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [36]:
max_length = 500 # Set a suitable maximum length for your input

# Truncate dialogue if it exceeds max_length
if len(dialogue) > max_length:
    dialogue = dialogue[:max_length]



In [ ]:
'''
dialogue
'''

In [11]:
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer_large, model = model_pega)

In [37]:
'''
pipe_out = pipe(dialogue)
pipe_outp = pipe_out[0]["summary_text"]
pipe_outp
'''

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
#pipe_outp

'anemia occurs in more than 50% of patients with cancer and may reach 90% in certain types of cancers , especially in patients undergoing chemotherapy and/or radiation therapy . <n> anemia often worsens symptoms such as fatigue , weakness , and dyspnea , and thus may have a negative effect on quality of life and performance status in patients with cancer . <n> this pilot study assessed the efficacy and feasibility of intravenous ( iv ) iron monotherapy in patients with cancer who have anemia and who are undergoing treatment with chemotherapy and/or radiation therapy without the use of erythropoiesis - stimulating agents . <n> eligible patients were at least 18 years old , about to start a cycle of chemotherapy and/or radiation therapy within 1 week of inclusion , and had a nonmyeloid malignancy , hemoglobin levels of 11.0 g / dl or less , a life expectancy of more than 24 weeks , and an eastern cooperative oncology group performance status of 02 . <n> patients were also required to hav

In [ ]:
#print(pipe_outp.replace(" . <n>", ".\n"))

anemia occurs in more than 50% of patients with cancer and may reach 90% in certain types of cancers , especially in patients undergoing chemotherapy and/or radiation therapy.
 anemia often worsens symptoms such as fatigue , weakness , and dyspnea , and thus may have a negative effect on quality of life and performance status in patients with cancer.
 this pilot study assessed the efficacy and feasibility of intravenous ( iv ) iron monotherapy in patients with cancer who have anemia and who are undergoing treatment with chemotherapy and/or radiation therapy without the use of erythropoiesis - stimulating agents.
 eligible patients were at least 18 years old , about to start a cycle of chemotherapy and/or radiation therapy within 1 week of inclusion , and had a nonmyeloid malignancy , hemoglobin levels of 11.0 g / dl or less , a life expectancy of more than 24 weeks , and an eastern cooperative oncology group performance status of 02.
 patients were also required to have a serum ferriti

In [ ]:
#summary

'backgroundanemia in patients with cancer who are undergoing active therapy is commonly encountered and may worsen quality of life in these patients . the effect of blood transfusion is often temporary and may be associated with serious adverse events . \n erythropoiesis - stimulating agents are not effective in 30%50% of patients and may have a negative effect on overall survival.aimsto assess the efficacy and feasibility of intravenous iron therapy in patients with cancer who have non - iron - deficiency anemia and who are undergoing treatment with chemotherapy without the use of erythropoiesis - stimulating agents.methodsadult patients with solid cancers and non - iron - deficiency anemia were included . \n ferric sucrose at a dose of 200 mg was given in short intravenous infusions weekly for a total of 12 weeks . \n hemoglobin level was measured at baseline , every 3 weeks , and 2 weeks after the last iron infusion ( week 14 ) . \n adverse events related to intravenous iron were pr

#### **Evaluating The Performance of Raw Model:**

In [12]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_TEST_ds(dataset, metric, model, tokenizer, batch_size=16, device=device, column_text="article", column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))
    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)
    return metric.compute()

In [13]:

rouge_metric = load("rouge")
'''
score = calculate_metric_on_TEST_ds(dataset_science["train"][0:10], rouge_metric, model_pega, tokenizer_large, batch_size=2, column_text="article", column_summary="abstract")
'''

'\nscore = calculate_metric_on_TEST_ds(dataset_science["train"][0:10], rouge_metric, model_pega, tokenizer_large, batch_size=2, column_text="article", column_summary="abstract")\n'

In [14]:

rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
'''
rouge_dict = {rn: score[rn] for rn in rouge_names}
print(pd.DataFrame(rouge_dict, index=["pegasus"]))
'''

'\nrouge_dict = {rn: score[rn] for rn in rouge_names}\nprint(pd.DataFrame(rouge_dict, index=["pegasus"]))\n'

#### **Fine-Tuning PEGASUS:**

##### **Tokenizing The Text:**

In [ ]:
tokens = tokenizer_large(dialogue, return_tensors="pt").input_ids
num_tokens = tokens.shape[1]
print(num_tokens)

Token indices sequence length is longer than the specified maximum sequence length for this model (4016 > 1024). Running this sequence through the model will result in indexing errors


4016


In [15]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer_large(example_batch["article"], max_length=1024, truncation=True)
    with tokenizer_large.as_target_tokenizer():
        target_encodings = tokenizer_large(example_batch["abstract"], max_length=128, truncation=True)
    return {
        "input_ids": input_encodings["input_ids"],
        "attention_mask": input_encodings["attention_mask"],
        "labels": target_encodings["input_ids"]
    }
dataset_science_pt = dataset_science.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/119924 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/6633 [00:00<?, ? examples/s]

Map:   0%|          | 0/6658 [00:00<?, ? examples/s]

In [ ]:
#dataset_science_pt["train"][0]

In [ ]:
'''
from datasets import DatasetDict, Dataset

# Helper function to chunk text
def chunk_article(article, max_chunk_length=1024):
    # Tokenize and chunk the article
    article_tokens = article.split()
    chunks = [' '.join(article_tokens[i:i + max_chunk_length])
              for i in range(0, len(article_tokens), max_chunk_length)]
    return chunks

# Function to apply chunking to the dataset
def chunk_dataset(dataset, max_chunk_length=1024):
    new_articles = []
    new_abstracts = []

    # Iterate over each example in the dataset
    for example in dataset:
        article_chunks = chunk_article(example['article'], max_chunk_length)
        for chunk in article_chunks:
            new_articles.append(chunk)
            new_abstracts.append(example['abstract'])  # Keep the same abstract for all chunks

    return Dataset.from_dict({'article': new_articles, 'abstract': new_abstracts})

# Apply chunking to each split
chunked_datasets = DatasetDict({
    'train': chunk_dataset(dataset_science['train']),
    'validation': chunk_dataset(dataset_science['validation']),
    'test': chunk_dataset(dataset_science['test'])
})

'''

In [16]:
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer_large, model = model_pega)

##### **Setting The Training Arguments:**

In [17]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir="pegasus-science", num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy="steps", eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)
'''
trainer_args = TrainingArguments(
    output_dir="./results",               # Output directory
    per_device_train_batch_size=4,        # Batch size for training
    per_device_eval_batch_size=4,         # Batch size for evaluation
    num_train_epochs=3,                   # Number of training epochs
    evaluation_strategy="epoch",          # Evaluate every epoch
    save_steps=10_000,                    # Save checkpoint every 10,000 steps
    save_total_limit=2,                   # Limit to how many checkpoints are saved
    #load_best_model_at_end=True,          # Load the best model at the end of training
    fp16=False                            # Disable mixed precision for now
)'''

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


'\ntrainer_args = TrainingArguments(\n    output_dir="./results",               # Output directory\n    per_device_train_batch_size=4,        # Batch size for training\n    per_device_eval_batch_size=4,         # Batch size for evaluation\n    num_train_epochs=3,                   # Number of training epochs\n    evaluation_strategy="epoch",          # Evaluate every epoch\n    save_steps=10_000,                    # Save checkpoint every 10,000 steps\n    save_total_limit=2,                   # Limit to how many checkpoints are saved\n    #load_best_model_at_end=True,          # Load the best model at the end of training\n    fp16=False                            # Disable mixed precision for now\n)'

In [ ]:
'''
model = model.to("cpu")
input_data = input_data.to("cpu")
labels = labels.to("cpu")
'''


NameError: name 'model' is not defined

##### **Training The Model:**

In [18]:
trainer = Trainer(model=model_pega, args=trainer_args,
                  tokenizer=tokenizer_large, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_science_pt["train"],
                  eval_dataset=dataset_science_pt["validation"])

In [ ]:
'''
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq

# Define trainer arguments
trainer_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_steps=10_000,
    save_total_limit=2,
    #load_best_model_at_end=True,
    fp16=False  # Disable mixed precision for now to debug CUDA issues
)

# Define data collator
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer_large, model=model_pega)

# Trainer instance
trainer = Trainer(
    model=model_pega,
    args=trainer_args,
    tokenizer=tokenizer_large,
    data_collator=seq2seq_data_collator,
    train_dataset=dataset_science_pt["train"],
    eval_dataset=dataset_science_pt["validation"]
)

# Start training
trainer.train()
'''

In [19]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
500,2.298700,2.116658
1000,2.316900,2.044609
1500,2.383400,1.998803
2000,2.152200,1.969721
2500,2.181000,1.944451
3000,2.203800,1.926405
3500,2.164700,1.907400
4000,2.142100,1.894241
4500,2.061300,1.883434
5000,2.147900,1.872984


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 256, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=7495, training_loss=2.189926869810383, metrics={'train_runtime': 56428.6993, 'train_samples_per_second': 2.125, 'train_steps_per_second': 0.133, 'total_flos': 3.3176376103378944e+17, 'train_loss': 2.189926869810383, 'epoch': 0.9999666455421767})

##### **Evaluating The Performance:**

In [20]:
score = calculate_metric_on_TEST_ds(dataset_science["test"], rouge_metric, model_pega, tokenizer_large, batch_size=2, column_text="article", column_summary="abstract")
rouge_dict = {rn: score[rn] for rn in rouge_names}
pd.DataFrame(rouge_dict, index=["pegasus"])

100%|██████████| 3329/3329 [3:28:37<00:00,  3.76s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.018248,0.000409,0.017046,0.01784


##### **Saving The Model:**

In [21]:
model_pega.save_pretrained("pegasus-science-model")

In [22]:
tokenizer_large.save_pretrained("pegasus-science-tokenizer")

('pegasus-science-tokenizer/tokenizer_config.json',
 'pegasus-science-tokenizer/special_tokens_map.json',
 'pegasus-science-tokenizer/spiece.model',
 'pegasus-science-tokenizer/added_tokens.json',
 'pegasus-science-tokenizer/tokenizer.json')